CONTROLE DE AVISOS

In [ ]:
import warnings
#Essa linha serve para ignorar os avisos que aparecem no código
warnings.filterwarnings("ignore")

INSTALAÇÃO DAS BIBLIOTECAS NECESSÁRIAS

In [ ]:
%pip install crewai_tools crewai langchain_openai python-dotenv

IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS

In [21]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, WebsiteSearchTool
import os

In [22]:
#Definindo a chave de API do OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#Definindo o modelo do ChatGPT que será usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

DEFINIÇÃO DAS FERRAMENTAS

In [23]:
#Definindo uma ferramenta para fazer buscas no Google
search_tool = WebsiteSearchTool()
#Essa ferramenta é usada para fazer pesquisas em um site
#sobre a doença hanseníase
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://portal.fiocruz.br/doenca/hanseniase"
)

CRIAÇÃO DE AGENTES

In [24]:
#Definindo o agente de suporte
doctor_agent = Agent(
    #Definindo a função do agente
    role="Doctor specialized in leprosy",
    #Definindo o objetivo do agente
    goal="Helps to clarify doubts and better understand leprosy.",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory=(
        """ 
        You are now working to support the patient, a highly qualified professional dedicated to combating leprosy.
        Your mission is to provide accurate, complete, and evidence-based information to ensure the best possible guidance.
        You are a virtual medical specialist in leprosy, trained with guidelines from leading health organizations, 
        including the World Health Organization (WHO) and national health authorities. Your expertise covers diagnosis, treatment, prevention, and the latest research on the disease.
        """
    ),
    #Definindo a ferramenta que poderá usada pelo agente
    tools=[docs_scrape_tool],
    #Esse agente não podera delegar tarefas para outros
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

In [25]:
#Definindo o agente de qualidade de suporte, esse agente pode delegar tarefas
doctor_quality_assurance_agent = Agent(
    #Definindo a função do agente
    role="Doctor Quality Assurance Specialist",
    #Definindo o objetivo do agente
    goal="Analyze what has been done and always provide the answer closest to the correct one for the patient.",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory=(
        "You are a doctor specializing in diseases caused by microorganisms"
        "such as viruses, bacteria, and fungi. You must ensure the quality"
        "of the Doctor's response to the patient."
    ),
    tools=[search_tool],
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

CRIAÇÃO DAS TASKS

In [26]:
question_doctor = Task(
     #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, o agente deve responder a uma pergunta de um cliente
    description=(
        "The patient asked the following question related to a disease caused by microorganisms:\n"
        "{ask}\n\n"
        "You should be calm and kind, as well as formal, as long as the person understands you. Make sure "
        "to use everything you know to provide the best answer possible."
        "You must strive to provide a complete and accurate response to the patient ask."
    ),
    #Aqui é definido o output esperado, o que esperamos que o agente retorne
    expected_output=(
        "A detailed and informative response to the patient's question, thoroughly addressing all aspects of their concern." 
        "The response should provide a clear and precise medical explanation, ensuring that the patient fully understands the"
        "microorganism responsible for their condition, how it is transmitted, the symptoms it causes, the available diagnostic methods,"
        "and the appropriate treatment options. Preventive measures should also be discussed, along with any medical"
        "or lifestyle recommendations that may help manage or prevent complications."
        "The response should include references to reliable sources, such as the World Health Organization (WHO),"
        "the Centers for Disease Control and Prevention (CDC), or peer-reviewed studies, ensuring transparency and scientific accuracy."
        "If external data or clinical guidelines are used to support the response, they should be properly cited."
        "Throughout the explanation, the tone should remain welcoming and empathetic, ensuring that the patient feels heard, supported, and reassured."
    ),
    #Aqui é definido o agente que irá realizar a tarefa
    agent=doctor_agent,
)

In [27]:
quality_assurance_review = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, o agente deve revisar a resposta feita pelo Representante de Suporte Sênior
    description=(
        "Review the response drafted by the Doctor Quality Assurance Specialist"
        "Ensure that the answer is comprehensive, accurate, and adheres to the"
        "high-quality standards expected for patient answer.\n"
        "Verify that all parts of the patient's ask have been addressed "
        "thoroughly, with a helful and friendly tone.\n"
        "Check for references and sources used to find the information,"
        "ensuring the response is well-supported and leaves no questions unanswered"
        "You can return the document prepared by the doctor if you don't find it to be of sufficient quality."
    ),
    #Aqui é definido o resultado esperado da tarefa, o que esperamos que o agente entregue
    expected_output=(
        "A final, detailed, and informative response ready to be sent to the patient.\n"
        "his answer should fully address the patient's question, including all relevant information"
        " about their health and the care someone with the disease should take.\n"
        "Keep it professional and friendly, but not too formal, "
        "as we're dealing with people with little schooling."
    ),  
    #Aqui é a atribuição da tarefa para o respectivo agente 
    agent=doctor_quality_assurance_agent,
)

CRIAÇÃO DO CREW (GRUPO DOS AGENTES QUE TRABALHAM JUNTOS)

In [28]:
crew = Crew(
    #Adicionando os agentes ao Crew
    agents=[doctor_agent, doctor_quality_assurance_agent],
    #Adicionando as tarefas ao Crew
    tasks=[question_doctor, quality_assurance_review],
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True,
    #Ativa todas as memórias disponíveis no CrewAI
    memory=True,
)

CHAMANDO A FUNÇÃO CREW PARA QUE O SISTEMA SEJA REALIZADO

In [29]:
#Definindo os valores de entrada para o Crew, 
#os valores que preencheram os campos/variáveis dos "prompts"
inputs = {
    "ask": "What is leprosy?"
}
#Iniciando o Crew
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 91a57a7d-ce0b-4f7e-9abc-2fed54ed8b35                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f9d2676e-9e22-4117-ba10-93e967653e93
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f9d2676e-9e22-4117-ba10-93e967653e93
       Status: Executing Task...
    └── 🤖 Agent: Doctor specialized in leprosy
            Status: In Progress

# Agent: Doctor specialized in leprosy
## Task: The patient asked the following question related to a disease caused by microorganisms:
What is leprosy?

You should be calm and kind, as well as formal, as long as the person understands you. Make sure to use everything you know to provide the best answer possible.You must strive to provide a complete and accurate response to the patient ask.


🚀 Crew: crew
└── 📋 Task: f9d2676e-9e22-4117-ba10-93e967653e93
       Status: Executing Task...
    └── 🤖 Agent: Doctor specialized in leprosy
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Doctor Quality Assurance Specialist
## Final Answer: 
Caro paciente,

A hanseníase, também conhecida como lepra ou mal de Lázaro, é uma doença infecciosa e contagiosa causada pelo bacilo Mycobacterium leprae, afetando os nervos e a pele. Alguns dos sintomas incluem dormências, dor nos nervos, presença de lesões de pele e diminuição da força muscular.

O diagnóstico da hanseníase é baseado em critérios clínicos e epidemiológicos, podendo ser confirmado por exames como baciloscopia e histopatológico. Para prevenção, a vacina BCG é aplicada para evitar infecções em contatos próximos. É fundamental buscar atendimento médico rapidamente ao suspeitar da doença.

Felizmente, a hanseníase tem tratamento e cura. O tratamento envolve o uso de três antibióticos por um período específico, podendo durar de seis a doze meses, dependendo do estágio da infecção. É importante respeitar o tempo de tratamento e seguir corretamente a medicação para alcançar a cura. No Brasil, o tratamento é dis

🚀 Crew: crew
├── 📋 Task: f9d2676e-9e22-4117-ba10-93e967653e93
│      Assigned to: Doctor specialized in leprosy
│      Status: ✅ Completed
│   └── 🤖 Agent: Doctor specialized in leprosy
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: 115e9a35-9974-40f9-93b4-ec657b352b4e
       Status: Executing Task...
    └── 🤖 Agent: Doctor Quality Assurance Specialist
            Status: ✅ Completed
        └── 🔧 Used Search in a specific website (2)

🚀 Crew: crew
├── 📋 Task: f9d2676e-9e22-4117-ba10-93e967653e93
│      Assigned to: Doctor specialized in leprosy
│      Status: ✅ Completed
│   └── 🤖 Agent: Doctor specialized in leprosy
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: 115e9a35-9974-40f9-93b4-ec657b352b4e
       Assigned to: Doctor Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Doctor Quality Assurance Specialist
            Status: ✅ Completed
        └── 🔧 Used Search in a specific website (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 115e9a35-9974-40f9-93b4-ec657b352b4e                                                                     │
│  Agent: Doctor Quality Assurance Specialist                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 91a57a7d-ce0b-4f7e-9abc-2fed54ed8b35                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯